# Graph Isomorphism Problem

The Graph Isomorphism Problem asks whether two given finite graphs are isomorphic.
An isomorphism of graphs $G_1$ and $G_2$ is a bijection (one-to-one mapping) between the vertex sets of $G_1$ and $G_2$,
such that the adjacency structure is preserved.

The Graph Isomorphism Problem and variations of it have many applications in areas such as:
- image processing
- protein structure
- computer systems
- malware detection
- chemical bond structure
- social networks

<table class="wikitable" style="margin: 1em auto 1em auto">
<tbody><tr>
<th>Graph G1
</th>
<th>Graph G2
</th>
<th>An isomorphism<br>between G1 and G2
</th></tr>
<tr>
<td style="padding-left:2em;padding-right:2em;"><a href="/wiki/File:Graph_isomorphism_a.svg" class="image"><img alt="Graph isomorphism a.svg" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Graph_isomorphism_a.svg/100px-Graph_isomorphism_a.svg.png" decoding="async" width="100" height="200" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Graph_isomorphism_a.svg/150px-Graph_isomorphism_a.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Graph_isomorphism_a.svg/200px-Graph_isomorphism_a.svg.png 2x" data-file-width="200" data-file-height="400"></a>
</td>
<td style="padding-left:1em;padding-right:1em;"><a href="/wiki/File:Graph_isomorphism_b.svg" class="image"><img alt="Graph isomorphism b.svg" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/84/Graph_isomorphism_b.svg/210px-Graph_isomorphism_b.svg.png" decoding="async" width="210" height="210" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/84/Graph_isomorphism_b.svg/315px-Graph_isomorphism_b.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/84/Graph_isomorphism_b.svg/420px-Graph_isomorphism_b.svg.png 2x" data-file-width="400" data-file-height="400"></a>
</td>
    <td align="center" style="background-color:white;">

<div style="padding:1em"><i>f</i>(<i>a</i>) = 1
</div><div style="padding:0.5em"><i>f</i>(<i>b</i>) = 6
</div><div style="padding:0.5em"><i>f</i>(<i>c</i>) = 8
</div><div style="padding:0.5em"><i>f</i>(<i>d</i>) = 3
</div><div style="padding:0.5em"><i>f</i>(<i>g</i>) = 5
</div><div style="padding:0.5em"><i>f</i>(<i>h</i>) = 2
</div><div style="padding:0.5em"><i>f</i>(<i>i</i>) = 4
</div><div style="padding:0.5em"><i>f</i>(<i>j</i>) = 7
</div>
</td></tr></tbody></table>

# Problem formulation - Objective and Constraints

## Decision Variables
Let's define a binary decision variable $x_{i,j}$ to determine if vertex $i$ of one graph ($G_1$) maps to vertex $j$ of another graph ($G_2$).

If $x_{i,j} = 1$, then we know $f(i) = j$ and $f^{-1}(j) = i$.

## Objective Function

We want to define the objective function (or Hamiltonian $H$) as a series of penalties for pairs of vertices that are connected in one graph and not in the other.

- Penalize all "bad edge mappings" (edge in $E_{G_1}$, but not in $E_{G_2}$, vice versa)

$$\Large H = \sum_{ij \notin E_{G_1}} \sum_{uv \in E_{G_2}} x_{i, u}x_{j,v}+\sum_{ij \in E_{G_1}} \sum_{uv \notin E_{G_2}} x_{i, u}x_{j,v}$$

For large, sparse graphs, this can produce a large number of biases because we are penalizing missing edges.


## Constraints

Based on the definition of the decision variable $x_{i,j}$, each vertex $i$ in graph $G_1$ can only be mapped to one vertex in the second graph and vice versa.

$$\Large\sum_{j \in V_{G_2}} x_{i,j} = 1 , \forall i \in G_1$$

$$\Large\sum_{i \in V_{G_1}} x_{i,j} = 1 , \forall j \in G_2 $$


## Discrete Quadratic Model (DQM)
Discrete quadratic model naturally handles one-hot constraints. We can think of different values of $j \in V_{G_2}$ as cases of each discrete variable $x_i$, such that if the binary representation $x_{i,j} = 1$, then the discrete representation is $x_i = j$. Conversely, $x_i = j$ implies not only that $x_{i,j}=1$ but also that all other $x_{i,k}=0$ if $k \neq j$.

# Code to solve an instance of the Graph Isomorphism Problem

In [ ]:
import networkx as nx

num_nodes = 10
graph_seed = 42

# Two graphs we know to be isomorphic by design
g_1 = nx.gnp_random_graph(n=num_nodes, p=0.5, seed=graph_seed)
g_1 = nx.relabel_nodes(g_1, {i: chr(i + ord('a')) for i in range(num_nodes)})

g_2 = nx.gnp_random_graph(n=num_nodes, p=0.5, seed=graph_seed)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
nx.draw(g_1, ax=ax[0], with_labels=True, font_color='w', font_size=18, node_size=500)
nx.draw(g_2, ax=ax[1], with_labels=True, font_color='w', font_size=18, node_size=500)
fig.tight_layout()

In [ ]:
from dimod import DQM

dqm = DQM()

for node in g_1.nodes:
    dqm.add_variable(num_nodes, label=node)

## Objective

In [ ]:
from itertools import combinations

penalty = 1
for (i, j) in g_1.edges:
    for u, v in combinations(g_2.nodes, r=2):
        if (u, v) not in g_2.edges:
            dqm.set_quadratic_case(i, u, j, v, penalty)
            dqm.set_quadratic_case(i, v, j, u, penalty)

for (u, v) in g_2.edges:
    for i, j in combinations(g_1.nodes, r=2):
        if (i, j) not in g_1.edges:
            dqm.set_quadratic_case(i, u, j, v, penalty)
            dqm.set_quadratic_case(i, v, j, u, penalty)

print('Total number of variables: ', dqm.num_variables())
print('Number of cases per variable: ', dqm.num_cases()//dqm.num_variables())
print('Total number of pairwise interactions: ', dqm.num_case_interactions())

## Constraints

Since the Discrete Quadratic Model naturally handles one-hot constraints, variables representing vertices in $G_1$ will only assume one case representing a vertex in $G_2$.

We still need to ensure that the variables representing each vertex in $G_1$ map to unique vertices in $G_2$, which is our bijection constraint above.

To encode this constraint, we must convert it into an objective: 
$$ \Large (\sum_{j \in V_{G_2}} x_{i,j} - 1)^2 $$

At this point we would expand the expression, however the Discrete Quadratic Model has a nice feature for these constraints.

In [ ]:
for u in g_2.nodes:
    dqm.add_linear_equality_constraint(
        [(i, u, 1.0) for i in g_1.nodes],
        constant=-1,
        lagrange_multiplier=penalty
    )

print('Total number of pairwise interactions: ', dqm.num_case_interactions())

print('Offset: ', dqm.offset)

## Solve with Leap Hybrid DQM Sampler

In [ ]:
from dwave.system import LeapHybridDQMSampler

sampler = LeapHybridDQMSampler()

sampleset = sampler.sample_dqm(dqm, time_limit=5)
sampleset.record.energy = dqm.energies(sampleset)

bijection = sampleset.first.sample
best_energy = sampleset.first.energy

print('Bijection: ', bijection)
print('Objective: ', best_energy)

In [ ]:
from utils import plot_graphs
plot_graphs(g_1, g_2, bijection)

### References and related demo

- Intro imagery and information about problem from: https://en.wikipedia.org/wiki/Graph_isomorphism

- D-Wave Examples circuit equivalence repository: https://github.com/dwave-examples/circuit-equivalence